In [32]:
%%capture

from lmi import lmi
import time
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
from tqdm.notebook import tqdm
import torch
from scipy.stats import kendalltau

os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]=".10"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"]="platform"
from bmi.estimators import MINEEstimator as MINE
from bmi.estimators import InfoNCEEstimator as InfoNCE

torch.manual_seed(2121)
np.random.seed(2121)

from scipy.special import erf

In [33]:
def half_cube(x: float) -> float:
    """The mapping x * sqrt(x), i.e., a signed version of x^(3/2) mapping."""
    return x * np.sqrt(np.abs(x))

def normal_cdf(x):
    return 0.5 * (1 + erf(x / 2**0.5))